In [17]:
# IMPORTS FOR FER
from tensorflow.keras.models import load_model
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
# IMPORTS FOR BODY POSTURE
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
#IMPORTS FOR EYE CONTACT
import cv2 as cv
import mediapipe as mp
import time
import utils, math
import numpy as np
#IMPORTS FOR RECORDING
import speech_recognition as sr
import sounddevice 
from scipy.io.wavfile import write 


# VARIABLES FOR FER
emotion_report = {'Angry':0, 'Disgust': 0, 'Fear':0, 'Happy':0, 'Neutral':0, 'Sad':0, 'Surprise':0}
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier =load_model(r'model.h5')
emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']
illegal_movements_count_face=0
cap = cv2.VideoCapture(0)
# VARIABLES FOR BODY POSTURE
illegal_movements_count_body = 0 
movement = None
# VARIABLES FOR EYE CONTACT
frame_counter =0
CEF_COUNTER =0
TOTAL_BLINKS =0
illegal_movements_count_eye = 0
start_time = time.time()
# constants
CLOSED_EYES_FRAME =3
FONTS =cv2.FONT_HERSHEY_COMPLEX
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]
# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]
map_face_mesh = mp.solutions.face_mesh
#VARIAVLES FOR RECORDING
count_text = []

# FUNCTIONS FOR BODY POSTURE 
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
# FUNCTIONS FOR EYE CONTACT
def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv2.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]
    # returning the list of tuples for each landmarks 
    return mesh_coord
# Euclaidean distance 
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)
    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]
    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]
    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)
    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)
    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance
    ratio = (reRatio+leRatio)/2
    return ratio 
# Eyes Extrctor function,
def eyesExtractor(img, right_eye_coords, left_eye_coords):
    # converting color image to  scale image 
    gray = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # getting the dimension of image 
    dim = gray.shape
    # creating mask from gray scale dim
    mask = np.zeros(dim, dtype=np.uint8)
    # drawing Eyes Shape on mask with white color 
    cv2.fillPoly(mask, [np.array(right_eye_coords, dtype=np.int32)], 255)
    cv2.fillPoly(mask, [np.array(left_eye_coords, dtype=np.int32)], 255)
    # showing the mask 
    # cv.imshow('mask', mask)
    
    # draw eyes image on mask, where white shape is 
    eyes = cv2.bitwise_and(gray, gray, mask=mask)
    # change black color to gray other than eys 
    # cv.imshow('eyes draw', eyes)
    eyes[mask==0]=155
    
    # getting minium and maximum x and y  for right and left eyes 
    # For Right Eye 
    r_max_x = (max(right_eye_coords, key=lambda item: item[0]))[0]
    r_min_x = (min(right_eye_coords, key=lambda item: item[0]))[0]
    r_max_y = (max(right_eye_coords, key=lambda item : item[1]))[1]
    r_min_y = (min(right_eye_coords, key=lambda item: item[1]))[1]
    # For LEFT Eye
    l_max_x = (max(left_eye_coords, key=lambda item: item[0]))[0]
    l_min_x = (min(left_eye_coords, key=lambda item: item[0]))[0]
    l_max_y = (max(left_eye_coords, key=lambda item : item[1]))[1]
    l_min_y = (min(left_eye_coords, key=lambda item: item[1]))[1]
    # croping the eyes from mask 
    cropped_right = eyes[r_min_y: r_max_y, r_min_x: r_max_x]
    cropped_left = eyes[l_min_y: l_max_y, l_min_x: l_max_x]
    # returning the cropped eyes 
    return cropped_right, cropped_left
# Eyes Postion Estimator 
def positionEstimator(cropped_eye):
    # getting height and width of eye 
    h, w =cropped_eye.shape
    
    # remove the noise from images
    gaussain_blur = cv2.GaussianBlur(cropped_eye, (9,9),0)

    # applying thresholding to convert binary_image
    ret, threshed_eye = cv2.threshold(gaussain_blur, 130, 255, cv.THRESH_BINARY)
    # create fixed part for eye with 
    piece = int(w/3) 
    # slicing the eyes into three parts 
    right_piece = threshed_eye[0:h, 0:piece]
    center_piece = threshed_eye[0:h, piece: piece+piece]
    left_piece = threshed_eye[0:h, piece +piece:w]
    
    # calling pixel counter function
    eye_position, color = pixelCounter(right_piece, center_piece, left_piece)
    return eye_position, color 

# creating pixel counter function 
def pixelCounter(first_piece, second_piece, third_piece):
    # counting black pixel in each part 
    right_part = np.sum(first_piece==0)
    center_part = np.sum(second_piece==0)
    left_part = np.sum(third_piece==0)
    # creating list of these values
    eye_parts = [right_part, center_part, left_part]
    # getting the index of max values in the list 
    max_index = eye_parts.index(max(eye_parts))
    pos_eye ='' 
    if max_index==0:
        pos_eye="RIGHT"
        color=[utils.BLACK, utils.WHITE]
    elif max_index==1:
        pos_eye = 'CENTER'
        color = [utils.BLACK, utils.WHITE]
    elif max_index ==2:
        pos_eye = 'LEFT'
        color = [utils.BLACK, utils.WHITE]
    else:
        pos_eye="Closed"
        color = [utils.BLACK, utils.WHITE]
    return pos_eye, color



#MAIN PROGRAM
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    with map_face_mesh.FaceMesh(min_detection_confidence =0.5, min_tracking_confidence=0.5) as face_mesh:
        while True:
            _, frame = cap.read()
            
            
            #FER
            labels = []
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray,1.2,4)
            # BODY POSTURE
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)# Recolor image to RGB
            image.flags.writeable = False
            results = pose.process(image) # Make detection
            image.flags.writeable = True # Recolor back to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)            
            #EYE CONTACT
            frame_eye = frame
            frame_eye = cv.resize(frame_eye, None, fx=1.5, fy=1.5, interpolation=cv.INTER_CUBIC)
            frame_height, frame_width= frame_eye.shape[:2]
            rgb_frame = cv.cvtColor(frame_eye, cv.COLOR_RGB2BGR)
            results_eye  = face_mesh.process(rgb_frame)
            
            
            #FER
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                roi_gray = gray[y:y+h,x:x+w]
                roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        
        
        
                if np.sum([roi_gray])!=0:
                    roi = roi_gray.astype('float')/255.0
                    roi = img_to_array(roi)
                    roi = np.expand_dims(roi,axis=0)
        
                    prediction = classifier.predict(roi)[0]
                    label=emotion_labels[prediction.argmax()]
                    emotion_report[label] += 1
                    if label=='Angry' or label=='Disgust' or  label == 'Fear' or label == 'Sad' or label == 'Surprise':
                                illegal_movements_count_face+=1
                    label_position = (x,y)
                    cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                else:
                    pass
                        
            # BODY POSTURE
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                lt_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                rt_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                # Calculate angle
                angle = calculate_angle(elbow, lt_shoulder, rt_shoulder)
                # illegal movement counter logic
                if angle > 96 and angle <130:
                    movement = "Straight :D"
                if angle>110:
                    illegal_movements_count_body +=1
                if angle < 96:   
                    illegal_movements_count_body +=1 
                
                # EYE CONTACT
                if results_eye.multi_face_landmarks:
                    mesh_coords = landmarksDetection(frame_eye, results_eye, False)
                    ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
                    if ratio >4:
                        CEF_COUNTER +=1
                    else:
                        if CEF_COUNTER>CLOSED_EYES_FRAME:
                            TOTAL_BLINKS +=1
                            CEF_COUNTER =0
                    # Blink Detector Counter Completed
                    right_coords = [mesh_coords[p] for p in RIGHT_EYE]
                    left_coords = [mesh_coords[p] for p in LEFT_EYE]
                    crop_right, crop_left = eyesExtractor(frame_eye, right_coords, left_coords)
                    eye_position, color = positionEstimator(crop_right)
                    eye_position_left, color = positionEstimator(crop_left)                    
                    #counting illegal eye movements
                    if eye_position == "RIGHT" or eye_position == "LEFT" or eye_position_left == "RIGHT" or eye_position_left == "LEFT":
                        illegal_movements_count_eye +=1
                # calculating  frame per seconds FPS
                end_time = time.time()-start_time
                fps = frame_counter/end_time  
            except:
                pass            
            with sr.Microphone() as source:
                r = sr.Recognizer()
                print("Start Speaking")
                audio = r.record(source, duration = 15)
                try:
                    text = r.recognize_google(audio)
                    count_text.append(text)
                    print("Speaker: ", text)
                    print("done")
                    if text == 'quit':
                        break
                except:
                    pass
            cv2.imshow('PROCTURING',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

Recording...
Done!
Recording...
Done!
Recording...
Done!


In [23]:
cap.release()

In [22]:
illegal_movements_count_eye

0

In [16]:
count_text

[]

In [1]:
# IMPORTS FOR FER
from tensorflow.keras.models import load_model
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
# IMPORTS FORR BODY POSTURE
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
#IMPORTS FOR EYE CONTACT
import cv2 as cv
import mediapipe as mp
import time
import utils, math
import numpy as np
#IMPORTS FOR RECORDING
import speech_recognition as sr
import sounddevice 
from scipy.io.wavfile import write 
#Record
import tkinter
from tkinter import messagebox


# VARIABLES FOR FER
emotion_report = {'Angry':0, 'Disgust': 0, 'Fear':0, 'Happy':0, 'Neutral':0, 'Sad':0, 'Surprise':0}
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier =load_model(r'model.h5')
emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']
illegal_movements_count_face=0
cap = cv2.VideoCapture(0)
# VARIABLES FOR BODY POSTURE
illegal_movements_count_body = 0 
movement = None
# VARIABLES FOR EYE CONTACT
frame_counter =0
CEF_COUNTER =0
TOTAL_BLINKS =0
illegal_movements_count_eye = 0
start_time = time.time()
# constants
CLOSED_EYES_FRAME =3
FONTS =cv2.FONT_HERSHEY_COMPLEX
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]
# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]
map_face_mesh = mp.solutions.face_mesh
#VARIAVLES FOR RECORDING
count_text = []

# FUNCTIONS FOR BODY POSTURE 
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
# FUNCTIONS FOR EYE CONTACT
def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv2.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]
    # returning the list of tuples for each landmarks 
    return mesh_coord
# Euclaidean distance 
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)
    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]
    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]
    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)
    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)
    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance
    ratio = (reRatio+leRatio)/2
    return ratio 
# Eyes Extrctor function,
def eyesExtractor(img, right_eye_coords, left_eye_coords):
    # converting color image to  scale image 
    gray = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # getting the dimension of image 
    dim = gray.shape
    # creating mask from gray scale dim
    mask = np.zeros(dim, dtype=np.uint8)
    # drawing Eyes Shape on mask with white color 
    cv2.fillPoly(mask, [np.array(right_eye_coords, dtype=np.int32)], 255)
    cv2.fillPoly(mask, [np.array(left_eye_coords, dtype=np.int32)], 255)
    # showing the mask 
    # cv.imshow('mask', mask)
    
    # draw eyes image on mask, where white shape is 
    eyes = cv2.bitwise_and(gray, gray, mask=mask)
    # change black color to gray other than eys 
    # cv.imshow('eyes draw', eyes)
    eyes[mask==0]=155
    
    # getting minium and maximum x and y  for right and left eyes 
    # For Right Eye 
    r_max_x = (max(right_eye_coords, key=lambda item: item[0]))[0]
    r_min_x = (min(right_eye_coords, key=lambda item: item[0]))[0]
    r_max_y = (max(right_eye_coords, key=lambda item : item[1]))[1]
    r_min_y = (min(right_eye_coords, key=lambda item: item[1]))[1]
    # For LEFT Eye
    l_max_x = (max(left_eye_coords, key=lambda item: item[0]))[0]
    l_min_x = (min(left_eye_coords, key=lambda item: item[0]))[0]
    l_max_y = (max(left_eye_coords, key=lambda item : item[1]))[1]
    l_min_y = (min(left_eye_coords, key=lambda item: item[1]))[1]
    # croping the eyes from mask 
    cropped_right = eyes[r_min_y: r_max_y, r_min_x: r_max_x]
    cropped_left = eyes[l_min_y: l_max_y, l_min_x: l_max_x]
    # returning the cropped eyes 
    return cropped_right, cropped_left
# Eyes Postion Estimator 
def positionEstimator(cropped_eye):
    # getting height and width of eye 
    h, w =cropped_eye.shape
    
    # remove the noise from images
    gaussain_blur = cv2.GaussianBlur(cropped_eye, (9,9),0)

    # applying thresholding to convert binary_image
    ret, threshed_eye = cv2.threshold(gaussain_blur, 130, 255, cv.THRESH_BINARY)
    # create fixed part for eye with 
    piece = int(w/3) 
    # slicing the eyes into three parts 
    right_piece = threshed_eye[0:h, 0:piece]
    center_piece = threshed_eye[0:h, piece: piece+piece]
    left_piece = threshed_eye[0:h, piece +piece:w]
    
    # calling pixel counter function
    eye_position, color = pixelCounter(right_piece, center_piece, left_piece)
    return eye_position, color 

# creating pixel counter function 
def pixelCounter(first_piece, second_piece, third_piece):
    # counting black pixel in each part 
    right_part = np.sum(first_piece==0)
    center_part = np.sum(second_piece==0)
    left_part = np.sum(third_piece==0)
    # creating list of these values
    eye_parts = [right_part, center_part, left_part]
    # getting the index of max values in the list 
    max_index = eye_parts.index(max(eye_parts))
    pos_eye ='' 
    if max_index==0:
        pos_eye="RIGHT"
        color=[utils.BLACK, utils.WHITE]
    elif max_index==1:
        pos_eye = 'CENTER'
        color = [utils.BLACK, utils.WHITE]
    elif max_index ==2:
        pos_eye = 'LEFT'
        color = [utils.BLACK, utils.WHITE]
    else:
        pos_eye="Closed"
        color = [utils.BLACK, utils.WHITE]
    return pos_eye, color
#FUNCTION FOR RECORDING
def record_audio():
        #It takes microphone input from the user and returns string output
        
        global count_text
        
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Listening...")
            #r.pause_threshold = 1
            #r.energy_threshold = 200  # minimum audio energy to consider for recording
            audio = r.listen(source)

        try:
            print("Recognizing...")    
            text = r.recognize_google(audio, language='en-in')
            count_text.append(text)
            print(f"Your Command: {text}\n")

        except Exception as e:
            print("Say that again please...") 
            return "None"
        return count_text

#FUNCTION FOR TEXT PROCESSING
def text_process(mess):
    ps = PorterStemmer()
    stemming = []
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    4. Returns stemming words from sentences
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    # Now just remove any stopwords
    # And stemming words from sentences
    nosw = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    for w in nosw:
        stemming.append(ps.stem(w))
    return stemming

def comparison(lst1, lst2):
    lst3 = [word for word in lst1 if word in lst2]
    return len(lst3)

#MAIN PROGRAM
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    with map_face_mesh.FaceMesh(min_detection_confidence =0.5, min_tracking_confidence=0.5) as face_mesh:
        with sr.Microphone() as source:
            while True:
                _, frame = cap.read()
                
                
                #FER
                labels = []
                gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                faces = face_classifier.detectMultiScale(gray,1.2,4)
                # BODY POSTURE
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)# Recolor image to RGB
                image.flags.writeable = False
                results = pose.process(image) # Make detection
                image.flags.writeable = True # Recolor back to BGR
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)            
                #EYE CONTACT
                frame_eye = frame
                frame_eye = cv.resize(frame_eye, None, fx=1.5, fy=1.5, interpolation=cv.INTER_CUBIC)
                frame_height, frame_width= frame_eye.shape[:2]
                rgb_frame = cv.cvtColor(frame_eye, cv.COLOR_RGB2BGR)
                results_eye  = face_mesh.process(rgb_frame)
                
                
                #FER
                for (x,y,w,h) in faces:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                    roi_gray = gray[y:y+h,x:x+w]
                    roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
            
            
            
                    if np.sum([roi_gray])!=0:
                        roi = roi_gray.astype('float')/255.0
                        roi = img_to_array(roi)
                        roi = np.expand_dims(roi,axis=0)
            
                        prediction = classifier.predict(roi)[0]
                        label=emotion_labels[prediction.argmax()]
                        emotion_report[label] += 1
                        if label=='Angry' or label=='Disgust' or  label == 'Fear' or label == 'Sad' or label == 'Surprise':
                                    illegal_movements_count_face+=1
                        label_position = (x,y)
                    else:
                        pass
                            
                # BODY POSTURE
                # Extract landmarks
                try:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get coordinates
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    lt_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    rt_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    
                    # Calculate angle
                    angle = calculate_angle(elbow, lt_shoulder, rt_shoulder)
                    # illegal movement counter logic
                    if angle > 96 and angle <130:
                        movement = "Straight :D"
                    if angle>110:
                        illegal_movements_count_body +=1
                    if angle < 96:   
                        illegal_movements_count_body +=1 
                    
                    # EYE CONTACT
                    if results_eye.multi_face_landmarks:
                        mesh_coords = landmarksDetection(frame_eye, results_eye, False)
                        ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
                        if ratio >4:
                            CEF_COUNTER +=1
                        else:
                            if CEF_COUNTER>CLOSED_EYES_FRAME:
                                TOTAL_BLINKS +=1
                                CEF_COUNTER =0
                        # Blink Detector Counter Completed
                        right_coords = [mesh_coords[p] for p in RIGHT_EYE]
                        left_coords = [mesh_coords[p] for p in LEFT_EYE]
                        crop_right, crop_left = eyesExtractor(frame_eye, right_coords, left_coords)
                        eye_position, color = positionEstimator(crop_right)
                        eye_position_left, color = positionEstimator(crop_left)                    
                        #counting illegal eye movements
                        if eye_position == "RIGHT" or eye_position == "LEFT" or eye_position_left == "RIGHT" or eye_position_left == "LEFT":
                            illegal_movements_count_eye +=1
                    # calculating  frame per seconds FPS
                    end_time = time.time()-start_time
                    fps = frame_counter/end_time
                    
                    #FOR RECORDING
                    r = sr.Recognizer()
                    
                    print("Listening...")
                    #r.pause_threshold = 1
                    #r.energy_threshold = 200  # minimum audio energy to consider for recording
                    audio = r.listen(source)
                    
                    print("Recognizing...")    
                    text = r.recognize_google(audio, language='en-in')
                    count_text.append(text)
                    print(f"Your Command: {text}\n")                 
                    
                except:
                    print("Say that again please...")
                    pass
                
    
                
                cv2.imshow('PROCTURING',frame)
                
                if (cv2.waitKey(1) & 0xFF == ord('q')) or count_text[-1]=='stop': 
                    break
            cap.release()
            cv2.destroyAllWindows()

Listening...
Recognizing...
Your Command: Python is general purpose programming language

Listening...
Recognizing...
Your Command: very nice language

Listening...
Recognizing...
Your Command: Gopi

Listening...
Recognizing...
Say that again please...
Listening...
Recognizing...
Say that again please...
Listening...
Recognizing...
Say that again please...
Listening...
Recognizing...
Your Command: hello

Listening...
Recognizing...
Say that again please...


In [2]:
count_text

['Python is general purpose programming language',
 'very nice language',
 'Gopi',
 'hello']

In [3]:
illegal_movements_count_face

1

In [1]:
!pip install pyttsx3

You should consider upgrading via the 'C:\Users\del\anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pyttsx3 #pip install pyttsx3
import speech_recognition as sr #pip install speechRecognition
import datetime

#Microsoft Speech API
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)

def speak(audio):
    print(" ")
    print(f"VAC: {audio} ")
    engine.say(audio)
    print("     ")
    engine.runAndWait()

def takeCommand():
        #It takes microphone input from the user and returns string output

        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Listening...")
            r.pause_threshold = 1
            r.energy_threshold = 200  # minimum audio energy to consider for recording
            audio = r.listen(source)

        try:
            print("Recognizing...")    
            query = r.recognize_google(audio, language='en-in')
            print(f"Your Command: {query}\n")

        except Exception as e:
            print("Say that again please...") 
            return "None"
        return query

def wishMe():
        hour = int(datetime.datetime.now().hour)
        if hour>=0 and hour<12:
            speak("Good Morning!")

        elif hour>=12 and hour<17:
            speak("Good Afternoon!")   

        else:
            speak("Good Evening!")  
        speak("Hi, I am VAC. Please tell me how may I help you?")

In [ ]:
# from process_module import speak
# from process_module import takeCommand
# from process_module import wishMe

def Task_Gui(self):
        wishMe()
        speak("Following are the commands you can use to speak with VAC")
        print("1.About\n2.Open College Website\n3.Courses\n4.Departments\n5.Facilities\n6.Contact\n7.Timing\n8.College Map\n9.Location\n10.Latest Updates\n11.Weather in GIT\n12.Fee Structure\n13.Exit")
        speak("Please ask anything from the given commands")
        print(" ")
        while True:
            self.query = takeCommand().lower()

            # Logic for executing tasks based on query
            if 'about' in self.query:
                speak('''Gharda Institute of Technology is established with a view to give quality education at affordable cost and the college started functioning in 2007 under the leadership of Padma Shri Dr. Keki Gharda. The college has aligned itself with the development of Konkan region and has contributed significantly by giving affordable education to students from konkan region.''')
            
            elif 'open college website' in self.query:
                speak("Here is G I T's college website")
                webbrowser.get('chrome').open("http://git-india.edu.in/git/index.html")  

            elif 'courses' in self.query:
                webbrowser.get('chrome').open("http://git-india.edu.in/git/academics_home.html")
                speak('''G I T provides engineering courses in the given fields
                    1st Bachelors in Chemical Engineering
                    2nd Bachelors in Mechanical Engineering
                    3rd Bachelors in Computer Engineering
                    4th Bachelors in Electronics and Telecommunication(EXTC) Engineering
                    5th Bachelors in Civil Engineering''')
            
            elif 'departments' in self.query:
                speak(''' There are 6 major departments in our college \n 1st Applied Science and Humanities \n 2nd Computer \n 3rd Chemical \n 4th Electronics and Telecommunications \n5th Mechanical\n 6th Civil''')
            
            elif 'facilities' in self.query:
                speak("Following are the facilities we have in our college")
                speak('''1) Computer Center\n2) eLearning Center\n3) Language Labs\n4) Medical Facility\n5) Transport Facility\n6) GymKhana (Gymnasium)\n7) Wifi & Video Conferencing\n8) Consumer Co Op Store\n9) Bank & Post ''')
            
            elif 'contact'in self.query:
                Contact()
                #speak("You can contact us by contacting the appropriate department you want to contact through the given details list")
                #webbrowser.get('chrome').open("http://git-india.edu.in/git/about_contactus.html")
            
            elif 'timing' in self.query:
                speak("College Timings are from morning 9 AM to Evening 5 PM")

            elif 'college map' in self.query:
                speak("Here is our college map")
                webbrowser.get('chrome').open("https://www.google.com/maps/place/Gharda+Institute+Of+Technology/@17.649687,73.4635664,174m/data=!3m1!1e3!4m5!3m4!1s0x3be9e1df45edd959:0xa318b89ca9383ae!8m2!3d17.6499538!4d73.4640929?hl=en")

            elif 'location' in self.query:
                speak("Please choose from below locations")
                Location()

            elif 'latest updates' in self.query:
                speak("Here are the latest updates happening in the college")
                webbrowser.get('chrome').open("http://git-india.edu.in/git/archives_news.asp")

            elif 'weather' in self.query:
                search = "Temperature in Lavel"
                url = f"https://www.google.com/search?q=temperature+in+lavel"
                r = requests.get(url)
                data = BeautifulSoup(r.text,"html.parser")
                temp = data.find("div",class_="BNeawe").text
                speak(f"Current {search} is {temp}")

            elif 'fee structure' in self.query:
                speak("Here is the FEE Structure")
                webbrowser.get('chrome').open("http://git-india.edu.in/git/pdf/Academics/FEES%202021-22.pdf")
    
            elif 'exit' in self.query:
                speak("It was great speaking to you, hope you have the best experience in G I T")
                exit()

In [1]:
name = 'Shabaz'
count = []


In [2]:
import pyttsx3 #pip install pyttsx3
import speech_recognition as sr #pip install speechRecognition
import datetime
def takeCommand():
        #It takes microphone input from the user and returns string output
        global count
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Listening...")
            r.pause_threshold = 1
#             r.energy_threshold = 200  # minimum audio energy to consider for recording
            audio = r.listen(source)

        try:
            print("Recognizing...")    
            query = r.recognize_google(audio, language='en-in')
            print(f"Your Command: {query}\n")
            count.append(query)
        except Exception as e:
            print("Say that again please...") 
            return "None"
        return count

In [3]:
takeCommand()

Listening...
Recognizing...
Your Command: hello



['hello']

In [12]:
!pip install SpeechRecognition

You should consider upgrading via the 'C:\Users\del\anaconda3\python.exe -m pip install --upgrade pip' command.


In [13]:
count

['hello Python general purpose programming language',
 'I love python',
 'hello you are very nice person']

In [ ]:
from django.shortcuts import render
from .models import QuestionsPython, Record, Solution, SolutionFace, SolutionBodyPosture, SolutionEyeContact
import random

#FOR RECORDING
import speech_recognition as sr
import pyaudio

# Specifically for FER
from tensorflow.keras.models import load_model
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
# import base64

#IMPORTS FOR BODY POSTURE
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#IMPORTS FOR eye_detection
import cv2 as cv
import mediapipe as mp
import time , math
import cipt_app.utils
import numpy as np

# IMPORTS FOR EYE DETECTION********************************
import cv2 as cv
import mediapipe as mp
import time
import second_phase.utils
import math
import numpy as np

#IMPORTS FOR BODY POSTURE DETECTION*************************
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#IMPORTS FOR FACE EMOTION DETECTION*************************
from tensorflow.keras.models import load_model
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np

#FOR RECORDING
import sounddevice
from scipy.io.wavfile import write

# imports for text processing
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import pandas as pd
import string
# Create your views here.

def tech_int_home(request):
    '''
    Here we inform user what is going to happen : Info page!!
    '''
    return render(request, 'phase2/tech_int_home.html')

def tech_int_select(request):
    '''
    Here candidate will select their topic of interest
    '''
    return render(request, 'phase2/tech_int_select.html')

def tech_int_python(request):
    '''
    Interview for Python
    '''
    ran_lst = [1,2,3,4,5,6]
    ran_int = random.choice(ran_lst)
    ques = QuestionsPython.objects.get(question_number=1)
    context = {
    'question' : ques
    }
    return render(request, 'phase2/tech_int_python.html',context)

def tech_int_ds(request):
    '''
    Interview for Python
    '''
    ran_lst = [1,2,3,4,5,6]
    ran_int = random.choice(ran_lst)
    ques = QuestionsDS.objects.get(question_number=1)
    context = {
    'question' : ques
    }
    return render(request, 'phase2/tech_int_python.html',context)

def tech_int_dbms(request):
    '''
    Interview for Python
    '''
    ran_lst = [1,2,3,4,5,6]
    ran_int = random.choice(ran_lst)
    ques = QuestionsDBMS.objects.get(question_number=1)
    context = {
    'question' : ques
    }
    return render(request, 'phase2/tech_int_python.html',context)

def tech_int_os(request):
    '''
    Interview for Python
    '''
    ran_lst = [1,2,3,4,5,6]
    ran_int = random.choice(ran_lst)
    ques = QuestionsOS.objects.get(question_number=1)
    context = {
    'question' : ques
    }
    return render(request, 'phase2/tech_int_python.html',context)

def tech_int_cn(request):
    '''
    Interview for Python
    '''
    ran_lst = [1,2,3,4,5,6]
    ran_int = random.choice(ran_lst)
    ques = QuestionsCN.objects.get(question_number=1)
    context = {
    'question' : ques
    }
    return render(request, 'phase2/tech_int_python.html',context)

# def ans_python(request):
#     return render(request, 'phase2/ans_python.html')


def record(request):
    #For recording the answers
    #VARIABLES FOR EYE DETECTION****************************************
    frame_counter =0
    CEF_COUNTER =0
    TOTAL_BLINKS =0
    illegal_movements_count_eye = 0
    start_time = time.time()

    # constants
    CLOSED_EYES_FRAME =3
    FONTS =cv2.FONT_HERSHEY_COMPLEX
    # face bounder indices
    FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
    # lips indices for Landmarks
    LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
    LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
    UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78]
    # Left eyes indices
    LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
    LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]
    # right eyes indices
    RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
    RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

    map_face_mesh = mp.solutions.face_mesh

    #VARIABLES FOR BODY POSTURE DETECTION*****************************
    illegal_movements_count_body = 0
    movement = None

    #VARIABLES FOR FACE EMOTION DETECTION*****************************
    emotion_report = {'Angry':0, 'Disgust': 0, 'Fear':0, 'Happy':0, 'Neutral':0, 'Sad':0, 'Surprise':0}
    illegal_movements_count_face = 0
    face_classifier = cv2.CascadeClassifier(r'second_phase/haarcascade_frontalface_default.xml')
    classifier =load_model(r'second_phase/model.h5')
    emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']




    #FUNCTION FOR EYE DETECTION*****************************************
    def landmarksDetection(img, results, draw=False):
        img_height, img_width= img.shape[:2]
        # list[(x,y), (x,y)....]
        mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
        if draw :
            [cv2.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]
        # returning the list of tuples for each landmarks
        return mesh_coord
    # Euclaidean distance
    def euclaideanDistance(point, point1):
        x, y = point
        x1, y1 = point1
        distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
        return distance
    # Blinking Ratio
    def blinkRatio(img, landmarks, right_indices, left_indices):
        # Right eyes
        # horizontal line
        rh_right = landmarks[right_indices[0]]
        rh_left = landmarks[right_indices[8]]
        # vertical line
        rv_top = landmarks[right_indices[12]]
        rv_bottom = landmarks[right_indices[4]]
        # draw lines on right eyes
        # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
        # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)
        # LEFT_EYE
        # horizontal line
        lh_right = landmarks[left_indices[0]]
        lh_left = landmarks[left_indices[8]]
        # vertical line
        lv_top = landmarks[left_indices[12]]
        lv_bottom = landmarks[left_indices[4]]
        rhDistance = euclaideanDistance(rh_right, rh_left)
        rvDistance = euclaideanDistance(rv_top, rv_bottom)
        lvDistance = euclaideanDistance(lv_top, lv_bottom)
        lhDistance = euclaideanDistance(lh_right, lh_left)
        reRatio = rhDistance/rvDistance
        leRatio = lhDistance/lvDistance
        ratio = (reRatio+leRatio)/2
        return ratio
    # Eyes Extrctor function,
    def eyesExtractor(img, right_eye_coords, left_eye_coords):
        # converting color image to  scale image
        gray = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)

        # getting the dimension of image
        dim = gray.shape
        # creating mask from gray scale dim
        mask = np.zeros(dim, dtype=np.uint8)
        # drawing Eyes Shape on mask with white color
        cv2.fillPoly(mask, [np.array(right_eye_coords, dtype=np.int32)], 255)
        cv2.fillPoly(mask, [np.array(left_eye_coords, dtype=np.int32)], 255)
        # showing the mask
        # cv.imshow('mask', mask)

        # draw eyes image on mask, where white shape is
        eyes = cv2.bitwise_and(gray, gray, mask=mask)
        # change black color to gray other than eys
        # cv.imshow('eyes draw', eyes)
        eyes[mask==0]=155

        # getting minium and maximum x and y  for right and left eyes
        # For Right Eye
        r_max_x = (max(right_eye_coords, key=lambda item: item[0]))[0]
        r_min_x = (min(right_eye_coords, key=lambda item: item[0]))[0]
        r_max_y = (max(right_eye_coords, key=lambda item : item[1]))[1]
        r_min_y = (min(right_eye_coords, key=lambda item: item[1]))[1]
        # For LEFT Eye
        l_max_x = (max(left_eye_coords, key=lambda item: item[0]))[0]
        l_min_x = (min(left_eye_coords, key=lambda item: item[0]))[0]
        l_max_y = (max(left_eye_coords, key=lambda item : item[1]))[1]
        l_min_y = (min(left_eye_coords, key=lambda item: item[1]))[1]
        # croping the eyes from mask
        cropped_right = eyes[r_min_y: r_max_y, r_min_x: r_max_x]
        cropped_left = eyes[l_min_y: l_max_y, l_min_x: l_max_x]
        # returning the cropped eyes
        return cropped_right, cropped_left
    # Eyes Postion Estimator
    def positionEstimator(cropped_eye):
        # getting height and width of eye
        h, w =cropped_eye.shape

        # remove the noise from images
        gaussain_blur = cv2.GaussianBlur(cropped_eye, (9,9),0)

        # applying thresholding to convert binary_image
        ret, threshed_eye = cv2.threshold(gaussain_blur, 130, 255, cv.THRESH_BINARY)
        # create fixed part for eye with
        piece = int(w/3)
        # slicing the eyes into three parts
        right_piece = threshed_eye[0:h, 0:piece]
        center_piece = threshed_eye[0:h, piece: piece+piece]
        left_piece = threshed_eye[0:h, piece +piece:w]

        # calling pixel counter function
        eye_position, color = pixelCounter(right_piece, center_piece, left_piece)
        return eye_position, color

    # creating pixel counter function
    def pixelCounter(first_piece, second_piece, third_piece):
        # counting black pixel in each part
        right_part = np.sum(first_piece==0)
        center_part = np.sum(second_piece==0)
        left_part = np.sum(third_piece==0)
        # creating list of these values
        eye_parts = [right_part, center_part, left_part]
        # getting the index of max values in the list
        max_index = eye_parts.index(max(eye_parts))
        pos_eye =''
        if max_index==0:
            pos_eye="RIGHT"
            color=[second_phase.utils.BLACK, second_phase.utils.WHITE]
        elif max_index==1:
            pos_eye = 'CENTER'
            color = [second_phase.utils.BLACK, second_phase.utils.WHITE]
        elif max_index ==2:
            pos_eye = 'LEFT'
            color = [second_phase.utils.BLACK, second_phase.utils.WHITE]
        else:
            pos_eye="Closed"
            color = [second_phase.utils.BLACK, second_phase.utils.WHITE]
        return pos_eye, color

    #FUNCTION FOR BODY POSTURE DETECTION****************************
    def calculate_angle(a,b,c):
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)

        if angle >180.0:
            angle = 360-angle

        return angle

    #FUNCTION FOR FACE DETECTION***************************************
    ###

    # FUNCTION FOR TEXT PROCESSING
    def text_process(mess):
        ps = PorterStemmer()
        stemming = []
        """
        Takes in a string of text, then performs the following:
        1. Remove all punctuation
        2. Remove all stopwords
        3. Returns a list of the cleaned text
        4. Returns stemming words from sentences
        """
        # Check characters to see if they are in punctuation
        nopunc = [char for char in mess if char not in string.punctuation]

        # Join the characters again to form the string.
        nopunc = ''.join(nopunc)

        # Now just remove any stopwords
        # And stemming words from sentences

        nosw = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

        for w in nosw:
            stemming.append(ps.stem(w))

        return stemming

    def comparison(lst1, lst2):
        lst3 = [word for word in lst1 if word in lst2]
        return len(lst3)

    #main program
    cap = cv2.VideoCapture(0)
    count = []
    with map_face_mesh.FaceMesh(min_detection_confidence =0.5, min_tracking_confidence=0.5) as face_mesh:
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
                #frame_counter +=1 # frame counter
                ret, frame = cap.read() # getting frame from camera
                #cv2.imshow("Procturing", frame)
                #FOR EYE DETECTION**************************************
                if not ret:
                    break # no more frames break
                #  resizing frame
                frame = cv2.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
                frame_height, frame_width= frame.shape[:2]
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                results  = face_mesh.process(rgb_frame)
                if results.multi_face_landmarks:
                    mesh_coords = landmarksDetection(frame, results, False)
                    ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
                    # cv.putText(frame, f'ratio {ratio}', (100, 100), FONTS, 1.0, utils.GREEN, 2)
                    #utils.colorBackgroundText(frame,  f'Ratio : {round(ratio,2)}', FONTS, 0.7, (30,100),2, utils.PINK, utils.YELLOW)
                    if ratio >4.1:
                        CEF_COUNTER +=1
                        # cv.putText(frame, 'Blink', (200, 50), FONTS, 1.3, utils.PINK, 2)
                        #utils.colorBackgroundText(frame,  f'Blink', FONTS, 1.7, (int(frame_height/2), 100), 2, utils.YELLOW, pad_x=6, pad_y=6, )
                    else:
                        if CEF_COUNTER>CLOSED_EYES_FRAME:
                            TOTAL_BLINKS +=1
                            CEF_COUNTER =0
                    # cv.putText(frame, f'Total Blinks: {TOTAL_BLINKS}', (100, 150), FONTS, 0.6, utils.GREEN, 2)
                    #utils.colorBackgroundText(frame,  f'Total Blinks: {TOTAL_BLINKS}', FONTS, 0.7, (30,150),2)

                    #cv2.polylines(frame,  [np.array([mesh_coords[p] for p in LEFT_EYE ], dtype=np.int32)], True, utils.GREEN, 1, cv.LINE_AA)
                    #cv2.polylines(frame,  [np.array([mesh_coords[p] for p in RIGHT_EYE ], dtype=np.int32)], True, utils.GREEN, 1, cv.LINE_AA)
                    # Blink Detector Counter Completed
                    right_coords = [mesh_coords[p] for p in RIGHT_EYE]
                    left_coords = [mesh_coords[p] for p in LEFT_EYE]
                    crop_right, crop_left = eyesExtractor(frame, right_coords, left_coords)
                    # cv.imshow('right', crop_right)
                    # cv.imshow('left', crop_left)
                    eye_position, color = positionEstimator(crop_right)
                    #utils.colorBackgroundText(frame, eye_position, FONTS, 1.0, (40, 220), 2, color[0], color[1], 8, 8)
                    #utils.colorBackgroundText(frame, f'R: {eye_position}', FONTS, 1.0, (40, 220), 2, color[0], color[1], 8, 8)
                    eye_position_left, color = positionEstimator(crop_left)
                    #utils.colorBackgroundText(frame, f'L: {eye_position_left}', FONTS, 1.0, (40, 320), 2, color[0], color[1], 8, 8)

                    #counting illegal eye movements
                    if eye_position == "RIGHT" or eye_position == "LEFT" or eye_position_left == "RIGHT" or eye_position_left == "LEFT":
                        illegal_movements_count_eye +=1


                #FOR BODY POSTURE*****************************************************
                #ret, frame = cap.read()

                # Recolor image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make detection
                results = pose.process(image)

                # Recolor back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract landmarks
                try:
                    landmarks = results.pose_landmarks.landmark

                    # Get coordinates
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    lt_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    rt_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                    # Calculate angle
                    angle = calculate_angle(elbow, lt_shoulder, rt_shoulder)

                    #Status box
                    cv2.rectangle(image, (0,0), (1000,55), (0,0,0), -1)
                    # Curl counter logic
                    if angle > 96 and angle <130:
                        movement = "Straight :D"
                    if angle>110:
                        movement = "Over Extended :("
                        #cv2.putText(image, "Seems that you've over extended your arms", (260,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

                        illegal_movements_count_body+=1
                    if angle < 96:
                        movement = "Cross :("
                        #cv2.putText(image, "Seems that you've crossed your arms", (260,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

                        illegal_movements_count_body +=1
                    #print(counter)

                except:
                    pass

                #FOR EMOTION RECOGNITION**********************************************
                labels = []
                gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                faces = face_classifier.detectMultiScale(gray,1.1,4)

                for (x,y,w,h) in faces:
                    #cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
                    roi_gray = gray[y:y+h,x:x+w]
                    roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



                    if np.sum([roi_gray])!=0:
                        roi = roi_gray.astype('float')/255.0
                        roi = img_to_array(roi)
                        roi = np.expand_dims(roi,axis=0)

                        prediction = classifier.predict(roi)[0]
                        label=emotion_labels[prediction.argmax()]
                        emotion_report[label] += 1

                        emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']
                        if label=='Angry' or label=='Disgust' or  label == 'Fear' or label == 'Sad' or label == 'Surprise':
                            illegal_movements_count_face+=1
                        label_position = (x,y)
                        #cv2.putText(image,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                    else:
                        pass
                        #cv2.putText(image,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)


                    with sr.Microphone() as source:

                        r = sr.Recognizer()
                        audio = r.record(source, duration = 15)

                        # fps = 44100
                        # duration = 15
                        # print("Recording...")
                        #recording = sounddevice.rec(int(duration*fps), samplerate=fps, channels=2)
                        #sounddevice.wait()
                        # print("Done!")
                        try:
                            text = r.recognize_google(audio)
                            count.append(text)
                            print("Speaker: ", text)
                        except:
                            pass



                    cv2.imshow("Procturing",frame)

                if 0xFF == ord('q') or cv2.waitKey(25000):# or text == "quit":
                    break

            df = pd.read_csv("second_phase/Interview_questions.csv", index_col='Index')
            df['Keywords'] = df['Answers'].apply(text_process)
            text = text_process(count)
            text_match_count = comparison(df['Keywords'][1], text)

            record_obj = Record(record_audio_text=count, illegal_face = illegal_movements_count_face, illegal_body = illegal_movements_count_body, illegal_eye = illegal_movements_count_eye,eye_blink_count = TOTAL_BLINKS, text_match = text_match_count) #record=audio,
            record_obj.save()
            cv.destroyAllWindows()
            cap.release()

        context = {
        'recording' : record_obj
        }
    return render(request, 'phase2/ans_python.html',context)

def ans_python(request):
    return render(request,'phase2/ans_python.html' )

def report(request):

    report = Record.objects.get(pk=1)
    #Variables
    ief = 0
    ieb = 0
    iee = 0
    tmp = 0
    tm = 0
    # for face
    if report.illegal_face <= 10:
        ief =100
    elif report.illegal_face > 10 and report.illegal_face <= 30:
        ief = 80
    elif report.illegal_face > 30 and report.illegal_face <= 50:
        ief = 60
    elif report.illegal_face > 50 and report.illegal_face <= 80:
        ief = 40
    else:
        ief = 20

    # for body
    if report.illegal_body <= 10:
        ieb = 100
    elif report.illegal_body > 10 and report.illegal_body <= 30:
        ieb = 80
    elif report.illegal_body > 30 and report.illegal_body <= 50:
        ieb = 60
    elif report.illegal_body > 50 and report.illegal_body <= 80:
        ieb = 40
    else:
        ieb = 20

    # for body
    if report.illegal_eye <= 10:
        iee = 100
    elif report.illegal_eye > 10 and report.illegal_eye <= 30:
        iee = 80
    elif report.illegal_eye > 30 and report.illegal_eye <= 50:
        iee = 60
    elif report.illegal_eye > 50 and report.illegal_eye <= 80:
        iee = 40
    else:
        iee = 20

    #FOR TECHNICAL INTERVIEW TEXT MATCH
    text_match = report.text_match

    if text_match < 2 :
        tmp = 80
    elif text_match >= 10:
        tm = 10
    else:
        tm = text_match
        tmp = 20

    tmu = tm * 10
    context = {
    'recording' : report,
    'ief' : ief,
    'ieb' : ieb,
    'iee' : iee,
    'tmu' : tmu,
    'tmp' : tmp
    }


    return render(request, 'phase2/report.html',context)


def illegal_face(request):
    videos = SolutionFace.objects.all()
    return render(request, 'phase2/illegal_face.html', context = {'videos' : videos})

def illegal_body(request):
    videos = SolutionBodyPosture.objects.all()
    return render(request, 'phase2/illegal_body.html', context = {'videos' : videos})

def illegal_eye(request):
    videos = SolutionEyeContact.objects.all()
    return render(request, 'phase2/illegal_eye.html', context = {'videos' : videos})

def tech_int_issue(request):
    videos = Solution.objects.all()
    return render(request, 'phase2/tech_int_issue.html', context = {'videos' : videos})



In [4]:
emotion_report = "{'Angry':0, 'Disgust': 0, 'Fear':7, 'Happy':34, 'Neutral':67, 'Sad':18, 'Surprise':8}"

In [11]:
emotion_report

{'Angry': 0,
 'Disgust': 0,
 'Fear': 7,
 'Happy': 34,
 'Neutral': 67,
 'Sad': 18,
 'Surprise': 8}

In [8]:
num


{'Angry': 0,
 'Disgust': 0,
 'Fear': 7,
 'Happy': 34,
 'Neutral': 67,
 'Sad': 18,
 'Surprise': 8}